# Purpose of notebook
This notebook is to continue to explore the data, in particular the time-series and bi-modal distributions from previous version. 

Topics covered:
- data normalization by round
- data cleaning/flattening


In [1]:
import json
import pandas as pd

## Loading the data

In [2]:
# Load JSON data from file
with open('pro2_data.json', 'r') as f:
    data = json.load(f)

# Access an element to retrieve the keys
sample_element = data[0]
columns = sample_element.keys()
print("Columns:", columns)

num_rows = len(data)
print("Number of rows:", num_rows)

Columns: dict_keys(['id', 'game_id', 'map_id', 'processed', 'process_date', 'created_at', 'updated_at', 'match_id', 'result', 'score', 'statistics', 'totals', 'ten_minute_averages', 'character_id', 'user_id', 'match_length', 'video_id', 'card_url', 'total_time', 'totals_to_date', 'start_time', 'end_time', 'start_frame', 'end_frame', 'status', 'report', 'analysis_processed', 'analysis_status'])
Number of rows: 448


## Create Rounds DF
- Create DF
- Go to statistics col

In [3]:
from pandas import json_normalize

# Create df
df = pd.read_json('pro2_data.json')
print(df.shape)

# Drop the all cols besides 'statistics' and 'user_id'
df_stats = df[['user_id','statistics']]
print(df_stats.shape)
df_stats

(448, 28)
(448, 2)


,user_id,statistics
0,3807,"{'map': 'Pearl', 'score': '6-13', 'allies': ['..."
1,3804,"{'map': 'unknown', 'score': '12-14', 'allies':..."
2,3806,"{'map': 'Split', 'score': '4-13', 'allies': ['..."
3,3808,"{'map': 'Haven', 'score': '8-13', 'allies': ['..."
4,3807,"{'map': 'Ascent', 'score': '13-8', 'allies': [..."
...,...,...
443,3799,"{'map': 'Ascent', 'score': '9-13', 'allies': [..."
444,3798,"{'map': 'Bind', 'score': '13-10', 'allies': ['..."
445,3795,"{'map': 'unknown', 'score': '16-14', 'allies':..."
446,3795,"{'map': 'unknown', 'score': '3-13', 'allies': ..."


In [5]:
df['statistics'].iloc[0]

{'map': 'Pearl',
 'score': '6-13',
 'allies': ['0', '1', '2', '3', '4'],
 'player': '2',
 'result': 'loss',
 'rounds': {'0': {'alive': {'0': [[89993, True],
     [134989, False],
     [140189, True],
     [172187, False]],
    '1': [[89993, True], [141989, False]],
    '2': [[89993, True], [156988, False]],
    '3': [[89993, True], [159787, False]],
    '4': [[89993, True], [161187, False]],
    '5': [[89993, True], [144789, False]],
    '6': [[89993, True], [156388, False]],
    '7': [[89993, True], [147988, False]],
    '8': [[89993, True], [159987, False]],
    '9': [[89993, True]]},
   'elims': [[147988,
     {'source': '4',
      'target': '7',
      'assisted': [],
      'headshot': True,
      'wallbang': False,
      'elim_number': 1,
      'first_blood': True,
      'team_credit': 'ally'}],
    [156388,
     {'source': '4',
      'target': '6',
      'assisted': ['2'],
      'headshot': True,
      'wallbang': False,
      'elim_number': 2,
      'first_blood': False,
      't

### Flattening the stats col by 1 level

In [4]:
flat1_df_stats = pd.json_normalize(df_stats['statistics'], max_level=0)
flat1_df_stats = pd.concat([df_stats['user_id'], flat1_df_stats], axis=1)
print(flat1_df_stats.shape)
flat1_df_stats

(448, 34)


,user_id,map,score,allies,player,result,rounds,status,totals,version,...,most_used_secondary,best_weapon_type_elims,most_used_primary_seconds,most_used_secondary_seconds,processed,allies_onscreen,analysis_status,detections_totals,analysis_processed,opponents_onscreen
0,3807,Pearl,6-13,"[0, 1, 2, 3, 4]",2,loss,"{'0': {'alive': {'0': [[89993, True], [134989,...",success: completed - valorant postprocess,"{'elims': 9, 'deaths': 14, 'assists': 3, 'heal...",1.0.0,...,sheriff,3,445,62,NaN,NaN,NaN,NaN,NaN,NaN
1,3804,unknown,12-14,"[0, 1, 2, 3, 4]",4,loss,"{'0': {'alive': {'0': [[3600, True], [105400, ...",success: completed - valorant postprocess,"{'elims': 20, 'deaths': 17, 'assists': 5, 'hea...",1.0.0,...,sheriff,10,1528,164,NaN,NaN,NaN,NaN,NaN,NaN
2,3806,Split,4-13,"[0, 1, 2, 3, 4]",2,loss,"{'0': {'alive': {'0': [[0, True], [41600, Fals...",success: completed - valorant postprocess,"{'elims': 13, 'deaths': 16, 'assists': 3, 'hea...",1.0.0,...,sheriff,6,659,115,NaN,NaN,NaN,NaN,NaN,NaN
3,3808,Haven,8-13,"[0, 1, 2, 3, 4]",4,loss,"{'0': {'alive': {'0': [[1200, True], [53800, F...",success: completed - valorant postprocess,"{'elims': 16, 'deaths': 12, 'assists': 4, 'hea...",1.0.0,...,ghost,8,475,96,NaN,NaN,NaN,NaN,NaN,NaN
4,3807,Ascent,13-8,"[0, 1, 2, 3, 4]",2,win,"{'0': {'alive': {'0': [[2999, True], [62795, F...",success: completed - valorant postprocess,"{'elims': 19, 'deaths': 10, 'assists': 15, 'he...",1.0.0,...,ghost,11,1082,256,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,3799,Ascent,9-13,"[0, 1, 2, 3, 4]",4,loss,"{'9': {'alive': {'0': [[0, True], [50200, Fals...",success: completed - valorant final,"{'elims': 10, 'deaths': 10, 'assists': 7, 'hea...",1.0.0,...,frenzy,7,338,176,True,"{'7150': 1, '14100': 2, '14200': 1, '16400': 0...",completed: processing - anyalysis (insights),"{'23': {'final_time': 16500, 'initial_size': {...",True,"{'7150': 0, '16400': 1, '20500': 0, '21850': 1..."
444,3798,Bind,13-10,"[0, 1, 2, 3, 4]",0,win,"{'0': {'alive': {'0': [[3999, True], [101999, ...",success: completed - valorant final,"{'elims': 25, 'deaths': 14, 'assists': 3, 'hea...",1.0.0,...,ghost,12,448,67,True,"{'5449': 0, '9649': 1, '46999': 2, '47849': 1,...",completed: processing - anyalysis (insights),"{'2': {'final_time': 5749, 'initial_size': {'h...",True,"{'5449': 1, '9649': 0, '88399': 1, '90699': 0,..."
445,3795,unknown,16-14,"[0, 1, 2, 3, 4]",1,win,"{'0': {'alive': {'0': [[75813, True], [151027,...",success: completed - valorant final,"{'elims': 25, 'deaths': 19, 'assists': 17, 'he...",1.0.0,...,ghost,10,1655,153,True,"{'33': 1, '79847': 2, '79981': 1, '80247': 2, ...",completed: processing - anyalysis (insights),"{'83': {'final_time': 125222, 'initial_size': ...",True,"{'33': 0, '125022': 1, '125056': 0, '125089': ..."
446,3795,unknown,3-13,"[0, 1, 2, 3, 4]",3,loss,"{'0': {'alive': {'0': [[3200, True], [75813, F...",success: completed - valorant final,"{'elims': 16, 'deaths': 13, 'assists': 0, 'hea...",1.0.0,...,ghost,8,280,205,True,"{'8901': 1, '14169': 0, '14202': 1, '39473': 2...",completed: processing - anyalysis (insights),"{'4': {'final_time': 14302, 'initial_size': {'...",True,"{'8901': 0, '14169': 1, '14202': 0, '14235': 1..."


### Removing some cols we don't care about from the ROUND perspective
- Keep in mind the 'totals', 'result' are from the MATCH perspective
- even though the map is from match perspective I need it so I am adding it in (similar to userID)
- weapon stuff is all from match perspective, so I am dropping (some exists in rounds data anyway)
- I think its possible to do something with 'allies_onscreen', 'opponents_onscreen', and 'detections_totals' but I am dropping for now


In [5]:
flat1_df_stats = flat1_df_stats.drop(['totals','score', 'allies','result', 'status','gametype','version','end_time','opponents','processed','ally_score','start_time','opponent_score','detections_totals','best_weapon_elims','most_used_primary_seconds','analysis_processed','most_used_secondary_seconds','best_weapon_type_elims','opponents_onscreen','allies_onscreen','analysis_status','player_totals','best_weapon_type','best_weapon','most_used_secondary','most_used_primary','best_weapon_type_elims'], axis=1)
print(flat1_df_stats.shape)
flat1_df_stats

(448, 7)


,user_id,map,player,rounds,player_ids,round_info,round_totals
0,3807,Pearl,2,"{'0': {'alive': {'0': [[89993, True], [134989,...","{'0': {'role': 'ally', 'character': 'gekko'}, ...","{'0': {'score': '0-0', 'ult_used': False, 'all...","{'0': {'elims': 0, 'deaths': 0, 'assists': 1, ..."
1,3804,unknown,4,"{'0': {'alive': {'0': [[3600, True], [105400, ...","{'0': {'role': 'ally', 'character': 'raze'}, '...","{'0': {'score': '0-0', 'ult_used': False, 'all...","{'0': {'elims': 1, 'deaths': 1, 'assists': 0, ..."
2,3806,Split,2,"{'0': {'alive': {'0': [[0, True], [41600, Fals...","{'0': {'role': 'ally', 'character': 'brimstone...","{'0': {'score': '0-0', 'ult_used': False, 'all...","{'0': {'elims': 0, 'deaths': 1, 'assists': 0, ..."
3,3808,Haven,4,"{'0': {'alive': {'0': [[1200, True], [53800, F...","{'0': {'role': 'ally', 'character': 'yoru'}, '...","{'0': {'score': '0-0', 'ult_used': False, 'all...","{'0': {'elims': 1, 'deaths': 1, 'assists': 0, ..."
4,3807,Ascent,2,"{'0': {'alive': {'0': [[2999, True], [62795, F...","{'0': {'role': 'ally', 'character': 'neon'}, '...","{'0': {'score': '0-0', 'ult_used': False, 'all...","{'0': {'elims': 2, 'deaths': 0, 'assists': 1, ..."
...,...,...,...,...,...,...,...
443,3799,Ascent,4,"{'9': {'alive': {'0': [[0, True], [50200, Fals...","{'0': {'role': 'ally', 'character': 'cypher'},...","{'9': {'score': '6-3', 'ult_used': False, 'all...","{'9': {'elims': 1, 'deaths': 0, 'assists': 1, ..."
444,3798,Bind,0,"{'0': {'alive': {'0': [[3999, True], [101999, ...","{'0': {'role': 'self', 'character': 'sage'}, '...","{'0': {'score': '0-0', 'ult_used': False, 'all...","{'0': {'elims': 1, 'deaths': 1, 'assists': 0, ..."
445,3795,unknown,1,"{'0': {'alive': {'0': [[75813, True], [151027,...","{'0': {'role': 'ally', 'character': 'raze'}, '...","{'0': {'score': '0-0', 'ult_used': False, 'all...","{'0': {'elims': 1, 'deaths': 0, 'assists': 0, ..."
446,3795,unknown,3,"{'0': {'alive': {'0': [[3200, True], [75813, F...","{'0': {'role': 'ally', 'character': 'omen'}, '...","{'0': {'score': '0-0', 'ult_used': False, 'all...","{'0': {'elims': 0, 'deaths': 1, 'assists': 0, ..."


### Flattening the player_ids col by 1 level
- this is getting all the teammates and opponents, as well as mapping the 'player' to an ally

In [6]:
# Create an empty list to hold the expanded player ids
expanded_player_ids_list = []

for i, player_ids in enumerate(flat1_df_stats['player_ids']):
    mapping = {}
    for player_id, player_info in player_ids.items():
        # If the role is 'self', map it to 'ally'
        role = 'ally' if player_info['role'] == 'self' else player_info['role']
        column_name = f"{role}{player_id}_character"
        mapping[column_name] = player_info['character']
    
    # If mapping is not empty
    if mapping:
        expanded_player_ids_list.append(mapping)
        
# Create the expanded_player_ids DataFrame
expanded_player_ids = pd.DataFrame(expanded_player_ids_list)

# Concatenate the original DataFrame with the expanded columns
flat2_df_stats_chars = pd.concat([flat1_df_stats.drop('player_ids', axis=1), expanded_player_ids], axis=1)

print(flat2_df_stats_chars.shape)
flat2_df_stats_chars

(448, 16)


,user_id,map,player,rounds,round_info,round_totals,ally0_character,ally1_character,ally2_character,ally3_character,ally4_character,opponent5_character,opponent6_character,opponent7_character,opponent8_character,opponent9_character
0,3807,Pearl,2,"{'0': {'alive': {'0': [[89993, True], [134989,...","{'0': {'score': '0-0', 'ult_used': False, 'all...","{'0': {'elims': 0, 'deaths': 0, 'assists': 1, ...",gekko,viper,phoenix,astra,jett,harbor,killjoy,jett,astra,yoru
1,3804,unknown,4,"{'0': {'alive': {'0': [[3600, True], [105400, ...","{'0': {'score': '0-0', 'ult_used': False, 'all...","{'0': {'elims': 1, 'deaths': 1, 'assists': 0, ...",raze,killjoy,sage,skye,omen,brimstone,raze,killjoy,jett,skye
2,3806,Split,2,"{'0': {'alive': {'0': [[0, True], [41600, Fals...","{'0': {'score': '0-0', 'ult_used': False, 'all...","{'0': {'elims': 0, 'deaths': 1, 'assists': 0, ...",brimstone,reyna,killjoy,jett,skye,omen,killjoy,jett,skye,raze
3,3808,Haven,4,"{'0': {'alive': {'0': [[1200, True], [53800, F...","{'0': {'score': '0-0', 'ult_used': False, 'all...","{'0': {'elims': 1, 'deaths': 1, 'assists': 0, ...",yoru,jett,chamber,killjoy,sova,jett,omen,killjoy,reyna,fade
4,3807,Ascent,2,"{'0': {'alive': {'0': [[2999, True], [62795, F...","{'0': {'score': '0-0', 'ult_used': False, 'all...","{'0': {'elims': 2, 'deaths': 0, 'assists': 1, ...",neon,sova,omen,breach,phoenix,killjoy,sova,jett,omen,reyna
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,3799,Ascent,4,"{'9': {'alive': {'0': [[0, True], [50200, Fals...","{'9': {'score': '6-3', 'ult_used': False, 'all...","{'9': {'elims': 1, 'deaths': 0, 'assists': 1, ...",cypher,astra,raze,sova,sage,kay/o,killjoy,omen,sova,jett
444,3798,Bind,0,"{'0': {'alive': {'0': [[3999, True], [101999, ...","{'0': {'score': '0-0', 'ult_used': False, 'all...","{'0': {'elims': 1, 'deaths': 1, 'assists': 0, ...",sage,raze,jett,chamber,reyna,phoenix,sova,raze,brimstone,chamber
445,3795,unknown,1,"{'0': {'alive': {'0': [[75813, True], [151027,...","{'0': {'score': '0-0', 'ult_used': False, 'all...","{'0': {'elims': 1, 'deaths': 0, 'assists': 0, ...",raze,viper,jett,brimstone,skye,raze,sova,jett,viper,brimstone
446,3795,unknown,3,"{'0': {'alive': {'0': [[3200, True], [75813, F...","{'0': {'score': '0-0', 'ult_used': False, 'all...","{'0': {'elims': 0, 'deaths': 1, 'assists': 0, ...",omen,jett,kay/o,killjoy,yoru,astra,jett,reyna,raze,sova


### Flattening the round_info/round_totals cols by 1 level
- This is where the DF expands into rounds being the rows instead of matches being the rows
- Have to parse the round info/round totals at the same time

In [7]:
# Initialize an empty dictionary to store map names and unique areas
map_areas = {}
for index, row in flat2_df_stats_chars.iterrows():
    # for all the rounds in each match
    for round_number, round_data in row['round_info'].items():
        # get the current/original row.
        new_row = row.to_dict()
        # exclude the original complex columns that we're flattening
        new_row.pop('rounds', None)
        new_row.pop('round_info', None)
        
        # get current map
        cur_map = new_row['map']

        # Initialize a new set for this map if it doesn't exist in map_areas
        if cur_map not in map_areas:
            map_areas[cur_map] = set()
            
        # Add round number to the new row
        new_row['round_number'] = round_number
        

        # Flatten rounds data into the new row
        for key, value in row['rounds'][round_number].items():
            if key == 'map_region':
                # value should be a list of lists where each sublist's second element is an area
                for sublist in value:
                    # Add the area to the set corresponding to the current map
                    map_areas[cur_map].add(sublist[1])

In [8]:
flat2_df_stats_chars.iloc[10:20]

,user_id,map,player,rounds,round_info,round_totals,ally0_character,ally1_character,ally2_character,ally3_character,ally4_character,opponent5_character,opponent6_character,opponent7_character,opponent8_character,opponent9_character
10,3806,Lotus,3,"{'0': {'alive': {'0': [[3600, True], [88200, F...","{'0': {'score': '0-0', 'ult_used': False, 'all...","{'0': {'elims': 0, 'deaths': 1, 'assists': 1, ...",killjoy,jett,reyna,skye,omen,jett,skye,breach,astra,cypher
11,3804,Lotus,4,"{'0': {'alive': {'0': [[3600, True], [73000, F...","{'0': {'score': '0-0', 'ult_used': False, 'all...","{'0': {'elims': 0, 'deaths': 1, 'assists': 1, ...",raze,brimstone,jett,killjoy,skye,omen,phoenix,raze,reyna,viper
12,3806,Ascent,3,"{'0': {'alive': {'0': [[53600, True], [131800,...","{'0': {'score': '0-0', 'ult_used': False, 'all...","{'0': {'elims': 0, 'deaths': 1, 'assists': 0, ...",omen,jett,fade,cypher,sage,jett,killjoy,omen,reyna,sova
13,3808,Lotus,3,"{'0': {'alive': {'0': [[2200, True], [71000, F...","{'0': {'score': '0-0', 'ult_used': False, 'all...","{'0': {'elims': 1, 'deaths': 1, 'assists': 0, ...",raze,omen,sova,skye,jett,chamber,omen,jett,raze,skye
14,3806,Haven,3,"{'0': {'alive': {'0': [[3400, True], [129400, ...","{'0': {'score': '0-0', 'ult_used': False, 'all...","{'0': {'elims': 0, 'deaths': 1, 'assists': 0, ...",jett,skye,viper,killjoy,omen,omen,killjoy,jett,sova,reyna
15,3804,unknown,1,"{'0': {'alive': {'0': [[3600, True], [80400, F...","{'0': {'score': '0-0', 'ult_used': False, 'all...","{'0': {'elims': 0, 'deaths': 0, 'assists': 2, ...",jett,omen,killjoy,sova,reyna,raze,jett,omen,killjoy,sova
16,3806,Lotus,0,"{'0': {'alive': {'0': [[118000, True], [191800...","{'0': {'score': '0-0', 'ult_used': False, 'all...","{'0': {'elims': 0, 'deaths': 1, 'assists': 1, ...",skye,cypher,reyna,jett,omen,omen,killjoy,breach,neon,jett
17,3806,Split,1,"{'0': {'alive': {'0': [[95400, True], [173200,...","{'0': {'score': '0-0', 'ult_used': False, 'all...","{'0': {'elims': 0, 'deaths': 0, 'assists': 0, ...",skye,astra,sage,cypher,raze,killjoy,omen,reyna,skye,sage
18,3806,Lotus,2,"{'0': {'alive': {'0': [[18000, True], [100200,...","{'0': {'score': '0-0', 'ult_used': False, 'all...","{'0': {'elims': 1, 'deaths': 0, 'assists': 1, ...",jett,omen,skye,chamber,killjoy,omen,skye,jett,killjoy,reyna
19,3808,Bind,4,"{'0': {'alive': {'0': [[39400, True], [157800,...","{'0': {'score': '0-0', 'ult_used': True, 'ally...","{'0': {'elims': 0, 'deaths': 1, 'assists': 0, ...",jett,omen,skye,reyna,sage,chamber,jett,harbor,skye,viper


In [9]:
def calculate_metrics(key, value, cur_player, new_row, spike_time):
    pre_spike_total = 0
    post_spike_total = 0
    pre_spike_entries = 0
    post_spike_entries = 0
    pre_spike_max_loss = 0
    post_spike_max_loss = 0
    pre_spike_total_loss = 0
    post_spike_total_loss = 0
    last_value = None

    for entry in value:
        entry_time = entry[0]
        entry_value = entry[1]

        if last_value is not None:
            value_loss = max(0, last_value - entry_value)
            if spike_time is None or entry_time < spike_time:
                pre_spike_max_loss = max(pre_spike_max_loss, value_loss)
                pre_spike_total_loss += value_loss
            elif spike_time:
                post_spike_max_loss = max(post_spike_max_loss, value_loss)
                post_spike_total_loss += value_loss

        last_value = entry_value

        if spike_time is None or entry_time < spike_time:
            pre_spike_total += entry_value
            pre_spike_entries += 1
        elif spike_time:
            post_spike_total += entry_value
            post_spike_entries += 1

    pre_spike_avg = pre_spike_total / pre_spike_entries if pre_spike_entries > 0 else 0
    post_spike_avg = post_spike_total / post_spike_entries if post_spike_entries > 0 else 0

    new_row[f'ally{cur_player}_pre_spike_avg_{key}'] = pre_spike_avg
    new_row[f'ally{cur_player}_post_spike_avg_{key}'] = post_spike_avg
    new_row[f'ally{cur_player}_pre_spike_max_{key}_loss'] = pre_spike_max_loss
    new_row[f'ally{cur_player}_post_spike_max_{key}_loss'] = post_spike_max_loss
    new_row[f'ally{cur_player}_pre_spike_total_{key}_loss'] = pre_spike_total_loss
    new_row[f'ally{cur_player}_post_spike_total_{key}_loss'] = post_spike_total_loss

In [10]:
def calculate_longest_duration(key, value, cur_player, new_row, spike_time):
    current_item = None
    longest_duration = 0
    longest_item = None
    pre_spike_longest_item = None
    post_spike_longest_item = None
    pre_spike_longest_duration = 0
    post_spike_longest_duration = 0

    for entry in value:
        timestamp = entry[0]
        item = entry[1]

        if current_item is None:
            current_item = item
            start_time = timestamp
        elif item != current_item:
            duration = timestamp - start_time
            if duration > longest_duration:
                longest_duration = duration
                longest_item = current_item

            if spike_time is None or start_time < spike_time:  # pre-spike or no spike
                if duration > pre_spike_longest_duration:
                    pre_spike_longest_duration = duration
                    pre_spike_longest_item = current_item
            elif spike_time and start_time >= spike_time:  # post-spike
                if duration > post_spike_longest_duration:
                    post_spike_longest_duration = duration
                    post_spike_longest_item = current_item

            current_item = item
            start_time = timestamp

    new_row[f'ally{cur_player}_longest_{key}'] = longest_item
    new_row[f'ally{cur_player}_pre_spike_longest_{key}'] = pre_spike_longest_item
    new_row[f'ally{cur_player}_post_spike_longest_{key}'] = post_spike_longest_item

In [11]:
def calculate_ability_usage(cur_player, ability_charges, spike_time, identifier, new_row):
    pre_spike_total_ability_usage = 0
    post_spike_total_ability_usage = 0

    for i in range(len(ability_charges) - 1):
        ability_time = ability_charges[i + 1][0]
        if ability_charges[i + 1][1] == 1:
            if spike_time is None or ability_time < spike_time:  # pre-spike or no spike
                pre_spike_total_ability_usage += 1
            elif spike_time:  # post-spike
                post_spike_total_ability_usage += 1

    new_row[f'ally{cur_player}_pre_spike_total_ability_usage_{identifier}'] = pre_spike_total_ability_usage
    new_row[f'ally{cur_player}_post_spike_total_ability_usage_{identifier}'] = post_spike_total_ability_usage

#     def calculate_ability_usage(cur_player, ability_charges, spike_time, identifier, new_row):
#     pre_spike_total_ability_usage = 0
#     post_spike_total_ability_usage = 0

#     for i in range(len(ability_charges) - 1):
#         ability_time = ability_charges[i + 1][0]
#         current_charge = ability_charges[i + 1][1]
#         previous_charge = ability_charges[i][1]

#         if previous_charge > current_charge:
#             if spike_time is None or ability_time < spike_time:  # pre-spike or no spike
#                 pre_spike_total_ability_usage += previous_charge - current_charge
#             elif spike_time:  # post-spike
#                 post_spike_total_ability_usage += previous_charge - current_charge

#     new_row[f'ally{cur_player}_pre_spike_total_ability_usage_{identifier}'] = pre_spike_total_ability_usage
#     new_row[f'ally{cur_player}_post_spike_total_ability_usage_{identifier}'] = post_spike_total_ability_usage


In [12]:
# First pass: get spike plant time for each round
spike_times = {}  # Create a dictionary to store the spike_time for each round
for index, row in flat2_df_stats_chars.iterrows():
    for round_number, round_data in row['round_info'].items():
        for key, value in row['rounds'][round_number].items():
            if key == 'spike_planted':
                # Initialize default values
                spike_yn = False
                time = 0
                rounds_spike_planted = value
                # Iterate over the list to find the first instance of spike planted
                for item in value:
                    if item[1]:  # Check if spike was planted
                        spike_yn = item[1]
                        time = item[0]
                        break  # Stop iterating after finding the first instance
   
                spike_times[(index, round_number)] = time if spike_yn else None

In [13]:
spike_times

{(0, '0'): None,
 (0, '1'): 257180,
 (0, '2'): None,
 (0, '3'): 471764,
 (0, '4'): 540359,
 (0, '5'): None,
 (0, '6'): None,
 (0, '7'): None,
 (0, '8'): 906931,
 (0, '9'): None,
 (0, '10'): 1045321,
 (0, '11'): 1122715,
 (0, '12'): None,
 (0, '13'): 1322700,
 (0, '14'): None,
 (0, '15'): None,
 (0, '16'): None,
 (0, '17'): 1655875,
 (0, '18'): 1774066,
 (1, '0'): 111800,
 (1, '1'): None,
 (1, '2'): 282600,
 (1, '3'): None,
 (1, '4'): None,
 (1, '5'): 605200,
 (1, '6'): None,
 (1, '7'): 812800,
 (1, '8'): None,
 (1, '9'): None,
 (1, '10'): 1075800,
 (1, '11'): 1174200,
 (1, '12'): 1273800,
 (1, '13'): 1376200,
 (1, '14'): 1494000,
 (1, '15'): 1570600,
 (1, '16'): None,
 (1, '17'): None,
 (1, '18'): 1860800,
 (1, '19'): None,
 (1, '20'): 2108600,
 (1, '21'): 2260200,
 (1, '22'): 2359400,
 (1, '23'): None,
 (1, '24'): 2569800,
 (1, '25'): None,
 (2, '0'): None,
 (2, '1'): None,
 (2, '2'): 201600,
 (2, '3'): 318000,
 (2, '4'): None,
 (2, '5'): None,
 (2, '6'): None,
 (2, '7'): 661600,
 (2,

In [14]:
new_rows = []

# Second pass: compute metrics
# for index, row in flat2_df_stats_chars.iterrows():
for index, row in flat2_df_stats_chars.iterrows():
    for round_number, round_data in row['round_info'].items():
        # get the current/original row.
        new_row = row.to_dict()
        # Get spike time for this round
        spike_time = spike_times[(index, round_number)]
        if spike_time:
            new_row['spike_planted'] = True
            new_row['spike_time'] = spike_time
        else:
            new_row['spike_planted'] = False
            new_row['spike_time'] = 0
        # get current map
        cur_map = new_row['map']
        # get current player
        cur_player = new_row['player']
        # Add round number to the new row
        new_row['round_number'] = round_number
        # Flatten round_info data into the new row
        for key, value in round_data.items():
            new_row[f'round_info_{key}'] = value
        # get side
        side = new_row['round_info_ally_side']
        # get won 
#             won = new_row['round_info_round_won']


        for key, value in row['rounds'][round_number].items():
            ## COUNTING ELIMINATIONS ##
            if key == 'elims':

                rounds_elims = value

                # Initialize the stats for all players
                player_stats = {str(player): {
                "pre_spike_elims": 0, "post_spike_elims": 0,
                "pre_spike_deaths": 0, "post_spike_deaths": 0,
                "pre_spike_assists": 0, "post_spike_assists": 0,
                "pre_spike_headshots": 0, "post_spike_headshots": 0,
                "pre_spike_wallbangs": 0, "post_spike_wallbangs": 0,
                "pre_spike_first_bloods": 0, "post_spike_first_bloods": 0} for player in range(10)}

                ally_deaths = 0
                opponent_deaths = 0

                # Then, for each elimination:
                for elimination in rounds_elims:
                    elim_data = elimination[1]
                    source = elim_data['source']
                    target = elim_data['target']
                    assisted = elim_data['assisted']

                    # check if the elimination happened before or after the spike
                    if spike_time is None or elimination[0] < spike_time:  # pre-spike
                        player_stats[source]['pre_spike_elims'] += 1
                        player_stats[target]['pre_spike_deaths'] += 1
                        for assist_player in assisted:
                            player_stats[assist_player]['pre_spike_assists'] += 1
                        if elim_data['headshot']:
                            player_stats[source]['pre_spike_headshots'] += 1
                        if elim_data['wallbang']:
                            player_stats[source]['pre_spike_wallbangs'] += 1
                        if elim_data['first_blood']:
                            player_stats[source]['pre_spike_first_bloods'] += 1
                    elif spike_time:  # post-spike, only if spike_time is not None
                        player_stats[source]['post_spike_elims'] += 1
                        player_stats[target]['post_spike_deaths'] += 1
                        for assist_player in assisted:
                            player_stats[assist_player]['post_spike_assists'] += 1
                        if elim_data['headshot']:
                            player_stats[source]['post_spike_headshots'] += 1
                        if elim_data['wallbang']:
                            player_stats[source]['post_spike_wallbangs'] += 1
                        if elim_data['first_blood']:
                            player_stats[source]['post_spike_first_bloods'] += 1

                    if int(target) < 5:  # If the target is an ally
                        ally_deaths += 1
                    else:  # If the target is an opponent
                        opponent_deaths += 1

                all_ally_dead = ally_deaths >= 5
                all_opponent_dead = opponent_deaths >= 5

                # Add these stats to the new_row, outside the eliminations loop:
                for player, stats in player_stats.items():
                    prefix = 'ally' if int(player) < 5 else 'opponent'
                    player_num = player if int(player) < 5 else str(int(player) - 5)
                    for stat, count in stats.items():
                        new_row[f'{prefix}{player_num}_{stat}'] = count

                new_row['all_ally_dead'] = all_ally_dead
                new_row['all_opponent_dead'] = all_opponent_dead

            ## CALC AVG HEALTH AND MAX/TOTAL HEALTH LOSS ##
            if key == 'health':
                rounds_health = value
                for player, health_data in rounds_health.items():
                    pre_spike_total_health = 0
                    post_spike_total_health = 0
                    pre_spike_entries = 0
                    post_spike_entries = 0
                    pre_spike_max_health_loss = 0
                    post_spike_max_health_loss = 0
                    pre_spike_total_health_loss = 0
                    post_spike_total_health_loss = 0
                    last_health = None
                    for entry in health_data:
                        health_time = entry[0]
                        health_value = entry[1]

                        if last_health is not None:  # if there is a previous health value, calculate the health loss
                            health_loss = max(0, last_health - health_value)  # don't let health loss be negative
                            if spike_time is None or health_time < spike_time:  # pre-spike or no spike
                                pre_spike_max_health_loss = max(pre_spike_max_health_loss, health_loss)
                                pre_spike_total_health_loss += health_loss
                            elif spike_time:  # post-spike
                                post_spike_max_health_loss = max(post_spike_max_health_loss, health_loss)
                                post_spike_total_health_loss += health_loss

                        last_health = health_value  # store the current health value as the last health value for the next iteration

                        if spike_time is None or health_time < spike_time:  # pre-spike or no spike
                            pre_spike_total_health += health_value
                            pre_spike_entries += 1
                        elif spike_time:  # post-spike
                            post_spike_total_health += health_value
                            post_spike_entries += 1

                    pre_spike_average_health = pre_spike_total_health / pre_spike_entries if pre_spike_entries > 0 else 0
                    post_spike_average_health = post_spike_total_health / post_spike_entries if post_spike_entries > 0 else 0

                    new_row[f'ally{player}_pre_spike_avg_health'] = pre_spike_average_health
                    new_row[f'ally{player}_post_spike_avg_health'] = post_spike_average_health
                    new_row[f'ally{player}_pre_spike_max_health_loss'] = pre_spike_max_health_loss
                    new_row[f'ally{player}_post_spike_max_health_loss'] = post_spike_max_health_loss
                    new_row[f'ally{player}_pre_spike_total_health_loss'] = pre_spike_total_health_loss
                    new_row[f'ally{player}_post_spike_total_health_loss'] = post_spike_total_health_loss
                    
            
            ## CALC AVG SHIELD AND MAX/TOTAL SHIELD LOSS ##
            if key == 'shield':
                calculate_metrics(key, value, cur_player, new_row, spike_time)
            ## CALC AVG CREDITS AND MAX/TOTAL CREDIT LOSS ##
            if key == 'credits':
                calculate_metrics(key, value, cur_player, new_row, spike_time)
            ## CALC AVG AMMO MAG AND MAX/TOTAL AMMO MAG LOSS ##  
            if key == 'ammo_mag':
                calculate_metrics(key, value, cur_player, new_row, spike_time)
            ## MOST TIME/MAX INV STATE ##
            if key == 'inv_state':
                calculate_longest_duration('inv_state', value, cur_player, new_row, spike_time)
            
            ## COUNTING ULTIMATE USAGE ##
            if key == 'ult_state':
                rounds_ult_state = value

                for player, ultimate_data in rounds_ult_state.items():
                    ultimate_usage = 0
                    pre_spike_ultimate_usage = 0
                    post_spike_ultimate_usage = 0
                    previous_state = None
                    for entry in ultimate_data:
                        ultimate_state_time = entry[0]
                        ultimate_state = entry[1]
                        if previous_state is not None and previous_state is True and ultimate_state is False:
                            ultimate_usage += 1
                            if spike_time is None or ultimate_state_time < spike_time:  # pre-spike or no spike
                                pre_spike_ultimate_usage += 1
                            elif spike_time and ultimate_state_time >= spike_time:  # post-spike
                                post_spike_ultimate_usage += 1
                        previous_state = ultimate_state
                    if int(player) < 5:
                        new_row[f'ally{player}_ultimate_usage'] = ultimate_usage
                        new_row[f'ally{player}_pre_spike_ultimate_usage'] = pre_spike_ultimate_usage
                        new_row[f'ally{player}_post_spike_ultimate_usage'] = post_spike_ultimate_usage
                    else:
                        new_row[f'opponent{int(player)-5}_ultimate_usage'] = ultimate_usage
                        new_row[f'opponent{int(player)-5}_pre_spike_ultimate_usage'] = pre_spike_ultimate_usage
                        new_row[f'opponent{int(player)-5}_post_spike_ultimate_usage'] = post_spike_ultimate_usage

            ## GETTING % MAP COVERED AND MOVEMENT METRIC"
            if key == 'map_region':
                rounds_map_region = value

                # % map covered
                available_places = map_areas[cur_map]
                visited_places_pre_spike = set()
                visited_places_post_spike = set()

                transitions_pre_spike = set()
                transitions_post_spike = set()

                previous_place_pre_spike = None
                previous_place_post_spike = None

                for entry in rounds_map_region:
                    timestamp = entry[0]
                    place = entry[1]

                    if spike_time is None or timestamp < spike_time:  # Pre spike or no spike planted
                        if place in available_places:
                            visited_places_pre_spike.add(place)

                        if previous_place_pre_spike is not None:
                            transition = (previous_place_pre_spike, place)
                            transitions_pre_spike.add(transition)

                        previous_place_pre_spike = place
                    else:  # Post spike
                        if place in available_places:
                            visited_places_post_spike.add(place)

                        if previous_place_post_spike is not None:
                            transition = (previous_place_post_spike, place)
                            transitions_post_spike.add(transition)

                        previous_place_post_spike = place

                metric_pre_spike = len(visited_places_pre_spike) / len(available_places)
                new_row[f'ally{cur_player}_pre_spike_map_covered'] = metric_pre_spike
                movement_metric_pre_spike = len(transitions_pre_spike)
                new_row[f'ally{cur_player}_pre_spike_movement_metric'] = movement_metric_pre_spike

                metric_post_spike = len(visited_places_post_spike) / len(available_places)
                new_row[f'ally{cur_player}_post_spike_map_covered'] = metric_post_spike
                movement_metric_post_spike = len(transitions_post_spike)
                new_row[f'ally{cur_player}_post_spike_movement_metric'] = movement_metric_post_spike

            
            ## MOST TIME/MAX PRIMARY GUN ##
            if key == 'inv_primary':
                calculate_longest_duration('gun_primary', value, cur_player, new_row, spike_time)
            ## CALC AVG AMMO RESERVE AND MAX/TOTAL AMMO RESERVE LOSS ##  
            if key == 'ammo_reserve':
                calculate_metrics(key, value, cur_player, new_row, spike_time)
            
            ## TOTAL FIRING TIME ##
            if key == 'firing_state':
                firing_state = value

                total_fire_time = 0
                pre_spike_total_fire_time = 0
                post_spike_total_fire_time = 0
                start_fire = 0
                firing = 0

                for i in range(len(firing_state) - 1):
                    if firing_state[i + 1][1] == True:
                        start_fire = firing_state[i + 1][0]
                    elif firing_state[i + 1][1] == False:
                        end_fire = firing_state[i + 1][0]
                        firing = end_fire - start_fire
                        total_fire_time += firing

                        if spike_time is None or start_fire < spike_time:  # pre-spike or no spike
                            pre_spike_total_fire_time += firing
                        elif spike_time and start_fire >= spike_time:  # post-spike
                            post_spike_total_fire_time += firing

                new_row[f'ally{cur_player}_total_firing_time'] = total_fire_time
                new_row[f'ally{cur_player}_pre_spike_total_firing_time'] = pre_spike_total_fire_time
                new_row[f'ally{cur_player}_post_spike_total_firing_time'] = post_spike_total_fire_time
            else:
                new_row[f'rounds_{key}'] = value   
            
            ## MOST TIME/MAX SECONDARY GUN ##
            if key == 'inv_secondary':
                calculate_longest_duration('gun_secondary', value, cur_player, new_row, spike_time)
            ## CALC AVG LOADOUT VALUE AND MAX/TOTAL LOADOUT VALUE LOSS ##
            if key == 'loadout_value':
                calculate_metrics(key, value, cur_player, new_row, spike_time)
            
            ## CALC ABILITY USE ##
            if key in ['ability_charges_1', 'ability_charges_2', 'ability_charges_3', 'ability_charges_4']:
                ability_charges = value
                identifier = key.split('_')[-1]  # extract the ability identifier from the key
                calculate_ability_usage(cur_player, ability_charges, spike_time, identifier, new_row)

        
        
        
        # Getting rid of cols that don't need 
        new_row.pop('rounds_alive', None)
        new_row.pop('rounds_phases', None)
        new_row.pop('rounds_spike_planted', None)
        new_row.pop('rounds_assisted_count', None)
        new_row.pop('rounds_assists_count', None)
        new_row.pop('rounds_elims_count', None)
        new_row.pop('round_totals', None)
        new_row.pop('round_info_ally_score', None)
        new_row.pop('round_info_opponent_score', None)

        # Getting rid of cols that we just parsed
        new_row.pop('rounds_elims', None)
        new_row.pop('rounds_health', None)
        new_row.pop('rounds_shield', None)
        new_row.pop('rounds_credits', None)
        new_row.pop('rounds_ammo_mag', None)
        new_row.pop('rounds_inv_state', None)
        new_row.pop('rounds_ult_state', None)
        new_row.pop('rounds_map_region', None)
        new_row.pop('rounds_inv_primary', None)
        new_row.pop('rounds_ammo_reserve', None)
        new_row.pop('rounds_firing_state', None)
        new_row.pop('rounds_inv_secondary', None)
        new_row.pop('rounds_loadout_value', None)
        new_row.pop('rounds_ability_charges_1', None)
        new_row.pop('rounds_ability_charges_2', None)
        new_row.pop('rounds_ability_charges_3', None)
        new_row.pop('rounds_ability_charges_4', None)
        
        # Getting rid of cols that we don't want/need    
        new_row.pop('round_info_score', None)
        new_row.pop('round_info_round_scored', None)
        new_row.pop('round_info_buy_start', None)
        new_row.pop('round_info_ult_used', None)
        new_row.pop('round_info_spike_planted', None)

        # exclude the original complex columns that we're flattening
        new_row.pop('rounds', None)
        new_row.pop('round_info', None)

        # fixing the naming of the players
        def adjust_keys(new_row, cur_player):
            adjusted_row = {}
            cur_player = int(cur_player)

            allies_count = 5  # define total number of allies, including 'self'

            for key, value in new_row.items():
                if key.startswith(f'ally{cur_player}'):
                    adjusted_key = key.replace(f'ally{cur_player}', 'self')
                elif key.startswith('ally'):
                    # correctly parse the ally number as an integer and remaining part of the key
                    ally_number = int(key[4:].split('_')[0])
                    ally_remaining = '_'.join(key.split('_')[1:])

                    # calculate new ally number
                    new_ally_number = (ally_number - cur_player) % allies_count

                    # construct new key with updated ally number
                    adjusted_key = 'ally' + str(new_ally_number) + '_' + ally_remaining
                else:
                    adjusted_key = key

                adjusted_row[adjusted_key] = value

            return adjusted_row

        new_row = adjust_keys(new_row, cur_player)

        # Append this new row to the list
        new_rows.append(new_row)

# Convert the list of new rows into a dataframe
new_df = pd.DataFrame(new_rows)


In [15]:
new_df

,user_id,map,player,ally3_character,ally4_character,self_character,ally1_character,ally2_character,opponent5_character,opponent6_character,...,self_pre_spike_total_loadout_value_loss,self_post_spike_total_loadout_value_loss,self_pre_spike_total_ability_usage_1,self_post_spike_total_ability_usage_1,self_pre_spike_total_ability_usage_2,self_post_spike_total_ability_usage_2,self_pre_spike_total_ability_usage_3,self_post_spike_total_ability_usage_3,self_pre_spike_total_ability_usage_4,self_post_spike_total_ability_usage_4
0,3807,Pearl,2,gekko,viper,phoenix,astra,jett,harbor,killjoy,...,1400.0,0.0,0,0,0,0,1,0,0,0
1,3807,Pearl,2,gekko,viper,phoenix,astra,jett,harbor,killjoy,...,1300.0,800.0,0,0,0,0,1,0,0,0
2,3807,Pearl,2,gekko,viper,phoenix,astra,jett,harbor,killjoy,...,1700.0,0.0,0,0,0,0,0,0,0,0
3,3807,Pearl,2,gekko,viper,phoenix,astra,jett,harbor,killjoy,...,2150.0,3900.0,1,0,0,0,0,0,0,0
4,3807,Pearl,2,gekko,viper,phoenix,astra,jett,harbor,killjoy,...,3790.0,1360.0,2,0,0,0,2,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9893,3795,unknown,0,reyna,breach,jett,skye,sova,jett,gekko,...,5800.0,0.0,1,0,0,0,1,0,0,0
9894,3795,unknown,0,reyna,breach,jett,skye,sova,jett,gekko,...,2800.0,0.0,1,0,0,0,1,0,0,0
9895,3795,unknown,0,reyna,breach,jett,skye,sova,jett,gekko,...,2000.0,1200.0,0,0,0,0,0,0,0,0
9896,3795,unknown,0,reyna,breach,jett,skye,sova,jett,gekko,...,2200.0,3000.0,1,0,0,0,0,0,1,0


### Data cleaning
- I noticed some NaNs. This happens when there is missing information
- self_longest_gun_primary, self_longest_inv_state, self_longest_gun_secondaryis coming up as None but i think it should be 'none', so i am replacing with that
- After investigating it seems like the elims data isnt always captured, its only 123 rows so im going to drop it

In [16]:
new_df['self_pre_spike_longest_gun_primary'] = new_df['self_pre_spike_longest_gun_primary'].fillna('none')
new_df['self_post_spike_longest_gun_primary'] = new_df['self_post_spike_longest_gun_primary'].fillna('none')
new_df['self_pre_spike_longest_gun_secondary'] = new_df['self_pre_spike_longest_gun_secondary'].fillna('none')
new_df['self_post_spike_longest_gun_secondary'] = new_df['self_post_spike_longest_gun_secondary'].fillna('none')
new_df['self_pre_spike_longest_inv_state'] = new_df['self_pre_spike_longest_inv_state'].fillna('none')
new_df['self_post_spike_longest_inv_state'] = new_df['self_post_spike_longest_inv_state'].fillna('none')
new_df['self_longest_inv_state'] = new_df['self_longest_inv_state'].fillna('none')
new_df['self_longest_gun_primary'] = new_df['self_longest_gun_primary'].fillna('none')
new_df['self_longest_gun_secondary'] = new_df['self_longest_gun_secondary'].fillna('none')

In [17]:
cols_with_nans = new_df.columns[new_df.isnull().any()].tolist()

# Print the list of columns with NaN values
cols_with_nans

[]

In [18]:
# Find rows with NaN values
nan_rows = new_df[new_df.isnull().any(axis=1)]

# Print the rows with NaN values
nan_rows

,user_id,map,player,ally3_character,ally4_character,self_character,ally1_character,ally2_character,opponent5_character,opponent6_character,...,self_pre_spike_total_loadout_value_loss,self_post_spike_total_loadout_value_loss,self_pre_spike_total_ability_usage_1,self_post_spike_total_ability_usage_1,self_pre_spike_total_ability_usage_2,self_post_spike_total_ability_usage_2,self_pre_spike_total_ability_usage_3,self_post_spike_total_ability_usage_3,self_pre_spike_total_ability_usage_4,self_post_spike_total_ability_usage_4


In [19]:
clean_df = new_df.dropna()
clean_df = clean_df.reset_index(drop=True)

In [20]:
# Find rows with NaN values
nan_rows = clean_df[clean_df.isnull().any(axis=1)]

# Print the rows with NaN values
nan_rows

,user_id,map,player,ally3_character,ally4_character,self_character,ally1_character,ally2_character,opponent5_character,opponent6_character,...,self_pre_spike_total_loadout_value_loss,self_post_spike_total_loadout_value_loss,self_pre_spike_total_ability_usage_1,self_post_spike_total_ability_usage_1,self_pre_spike_total_ability_usage_2,self_post_spike_total_ability_usage_2,self_pre_spike_total_ability_usage_3,self_post_spike_total_ability_usage_3,self_pre_spike_total_ability_usage_4,self_post_spike_total_ability_usage_4


In [21]:
clean_df.shape

(9898, 256)

In [22]:
clean_df.columns[:50]

Index(['user_id', 'map', 'player', 'ally3_character', 'ally4_character',
       'self_character', 'ally1_character', 'ally2_character',
       'opponent5_character', 'opponent6_character', 'opponent7_character',
       'opponent8_character', 'opponent9_character', 'spike_planted',
       'spike_time', 'round_number', 'round_info_ally_side',
       'round_info_round_end', 'round_info_round_won',
       'round_info_round_start', 'ally3_pre_spike_elims',
       'ally3_post_spike_elims', 'ally3_pre_spike_deaths',
       'ally3_post_spike_deaths', 'ally3_pre_spike_assists',
       'ally3_post_spike_assists', 'ally3_pre_spike_headshots',
       'ally3_post_spike_headshots', 'ally3_pre_spike_wallbangs',
       'ally3_post_spike_wallbangs', 'ally3_pre_spike_first_bloods',
       'ally3_post_spike_first_bloods', 'ally4_pre_spike_elims',
       'ally4_post_spike_elims', 'ally4_pre_spike_deaths',
       'ally4_post_spike_deaths', 'ally4_pre_spike_assists',
       'ally4_post_spike_assists', 'ally

In [23]:
clean_df.columns[50:100]

Index(['self_pre_spike_headshots', 'self_post_spike_headshots',
       'self_pre_spike_wallbangs', 'self_post_spike_wallbangs',
       'self_pre_spike_first_bloods', 'self_post_spike_first_bloods',
       'ally1_pre_spike_elims', 'ally1_post_spike_elims',
       'ally1_pre_spike_deaths', 'ally1_post_spike_deaths',
       'ally1_pre_spike_assists', 'ally1_post_spike_assists',
       'ally1_pre_spike_headshots', 'ally1_post_spike_headshots',
       'ally1_pre_spike_wallbangs', 'ally1_post_spike_wallbangs',
       'ally1_pre_spike_first_bloods', 'ally1_post_spike_first_bloods',
       'ally2_pre_spike_elims', 'ally2_post_spike_elims',
       'ally2_pre_spike_deaths', 'ally2_post_spike_deaths',
       'ally2_pre_spike_assists', 'ally2_post_spike_assists',
       'ally2_pre_spike_headshots', 'ally2_post_spike_headshots',
       'ally2_pre_spike_wallbangs', 'ally2_post_spike_wallbangs',
       'ally2_pre_spike_first_bloods', 'ally2_post_spike_first_bloods',
       'opponent0_pre_spike_elims'

In [24]:
clean_df.columns[100:150]

Index(['opponent1_pre_spike_wallbangs', 'opponent1_post_spike_wallbangs',
       'opponent1_pre_spike_first_bloods', 'opponent1_post_spike_first_bloods',
       'opponent2_pre_spike_elims', 'opponent2_post_spike_elims',
       'opponent2_pre_spike_deaths', 'opponent2_post_spike_deaths',
       'opponent2_pre_spike_assists', 'opponent2_post_spike_assists',
       'opponent2_pre_spike_headshots', 'opponent2_post_spike_headshots',
       'opponent2_pre_spike_wallbangs', 'opponent2_post_spike_wallbangs',
       'opponent2_pre_spike_first_bloods', 'opponent2_post_spike_first_bloods',
       'opponent3_pre_spike_elims', 'opponent3_post_spike_elims',
       'opponent3_pre_spike_deaths', 'opponent3_post_spike_deaths',
       'opponent3_pre_spike_assists', 'opponent3_post_spike_assists',
       'opponent3_pre_spike_headshots', 'opponent3_post_spike_headshots',
       'opponent3_pre_spike_wallbangs', 'opponent3_post_spike_wallbangs',
       'opponent3_pre_spike_first_bloods', 'opponent3_post_spi

In [25]:
clean_df.columns[150:200]

Index(['ally4_pre_spike_max_health_loss', 'ally4_post_spike_max_health_loss',
       'ally4_pre_spike_total_health_loss',
       'ally4_post_spike_total_health_loss', 'self_pre_spike_avg_health',
       'self_post_spike_avg_health', 'self_pre_spike_max_health_loss',
       'self_post_spike_max_health_loss', 'self_pre_spike_total_health_loss',
       'self_post_spike_total_health_loss', 'ally1_pre_spike_avg_health',
       'ally1_post_spike_avg_health', 'ally1_pre_spike_max_health_loss',
       'ally1_post_spike_max_health_loss', 'ally1_pre_spike_total_health_loss',
       'ally1_post_spike_total_health_loss', 'ally2_pre_spike_avg_health',
       'ally2_post_spike_avg_health', 'ally2_pre_spike_max_health_loss',
       'ally2_post_spike_max_health_loss', 'ally2_pre_spike_total_health_loss',
       'ally2_post_spike_total_health_loss', 'self_pre_spike_avg_shield',
       'self_post_spike_avg_shield', 'self_pre_spike_max_shield_loss',
       'self_post_spike_max_shield_loss', 'self_pre_spi

In [26]:
clean_df.columns[200:]

Index(['self_pre_spike_ultimate_usage', 'self_post_spike_ultimate_usage',
       'ally1_ultimate_usage', 'ally1_pre_spike_ultimate_usage',
       'ally1_post_spike_ultimate_usage', 'ally2_ultimate_usage',
       'ally2_pre_spike_ultimate_usage', 'ally2_post_spike_ultimate_usage',
       'opponent0_ultimate_usage', 'opponent0_pre_spike_ultimate_usage',
       'opponent0_post_spike_ultimate_usage', 'opponent1_ultimate_usage',
       'opponent1_pre_spike_ultimate_usage',
       'opponent1_post_spike_ultimate_usage', 'opponent2_ultimate_usage',
       'opponent2_pre_spike_ultimate_usage',
       'opponent2_post_spike_ultimate_usage', 'opponent3_ultimate_usage',
       'opponent3_pre_spike_ultimate_usage',
       'opponent3_post_spike_ultimate_usage', 'opponent4_ultimate_usage',
       'opponent4_pre_spike_ultimate_usage',
       'opponent4_post_spike_ultimate_usage', 'self_pre_spike_map_covered',
       'self_pre_spike_movement_metric', 'self_post_spike_map_covered',
       'self_post_spik

In [27]:
clean_df.to_csv("pro2_data_norm_round_spikesplit.csv", index=False)